<a href="https://colab.research.google.com/github/caseykotowski/Song-Features-Over-Time/blob/DB_Segment2_MCC/MCC_Loading_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [66.2 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("LoadingData").getOrCreate()

In [3]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://spot-i-fy-bucket.s3.us-east-2.amazonaws.com/tracks.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("tracks.csv"), sep=",", header=True)

In [4]:
# Show DataFrame
df.show()

+--------------------+--------------------+----------+-----------+--------+-------------------+--------------------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+
|                  id|                name|popularity|duration_ms|explicit|            artists|          id_artists|release_date|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|time_signature|
+--------------------+--------------------+----------+-----------+--------+-------------------+--------------------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+
|35iwgR4jXetI318WE...|               Carve|         6|     126903|       0|            ['Uli']|['45tIt06XoI0Iio4...|  1922-02-22|       0.645| 0.445|  0| -13.338|   1|      0.451|       0.674|           0.744|   0.151|  0.127|104.851|             3|


In [12]:
# Print our schema
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- duration_ms: string (nullable = true)
 |-- explicit: string (nullable = true)
 |-- artists: string (nullable = true)
 |-- id_artists: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- danceability: string (nullable = true)
 |-- energy: string (nullable = true)
 |-- key: string (nullable = true)
 |-- loudness: string (nullable = true)
 |-- mode: string (nullable = true)
 |-- speechiness: string (nullable = true)
 |-- acousticness: string (nullable = true)
 |-- instrumentalness: string (nullable = true)
 |-- liveness: string (nullable = true)
 |-- valence: string (nullable = true)
 |-- tempo: string (nullable = true)
 |-- time_signature: string (nullable = true)



In [13]:
print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())

586589
586589
586589


In [14]:
# Create the products_table DataFrame and drop duplicates. 
from pyspark.sql.functions import to_date

tracks_df = df.select(["id", "name", "popularity", "duration_ms", "explicit", 
                       "id_artists", "danceability", "energy", "loudness", "speechiness", 
                       "acousticness", "instrumentalness", "liveness", "valence", "tempo", 
                       "time_signature", "release_date"])
tracks_df.show(10)

+--------------------+--------------------+----------+-----------+--------+--------------------+------------+------+--------+-----------+------------+----------------+--------+-------+-------+--------------+------------+
|                  id|                name|popularity|duration_ms|explicit|          id_artists|danceability|energy|loudness|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|time_signature|release_date|
+--------------------+--------------------+----------+-----------+--------+--------------------+------------+------+--------+-----------+------------+----------------+--------+-------+-------+--------------+------------+
|25SMQZnHTtKcUxFIp...|The Loveliest Nig...|         0|     162064|       0|['3E3AwPbm9ShPKXV...|       0.433| 0.422| -12.455|     0.0418|       0.815|        9.67e-06|   0.114|  0.453|160.021|             4|        1923|
|2XPuwjQubVd5zfWmB...|Chapter 9.20 & Ch...|         0|     187500|       0|['4hLtcEEpjCmBtTn...|       0.642| 0.215|

In [15]:
artists_df = df.select(["id_artists", "artists"])
artists_df.show(10)

+--------------------+--------------------+
|          id_artists|             artists|
+--------------------+--------------------+
|['3E3AwPbm9ShPKXV...|    ['Harry Curtis']|
|['4hLtcEEpjCmBtTn...|  ['Janusz Korczak']|
|['4hLtcEEpjCmBtTn...|  ['Janusz Korczak']|
|['7Ir39wGyJWCQ9gm...|['4509 STARPASS P...|
|['2OJEUmIIPsTAUji...|['Radclyffe Hall'...|
|['6UftTZlUzZBBl2c...|['Lionel Hampton ...|
|['5LiOoJbxVSAMkBS...| ['Ignacio Corsini']|
|['2maQMqxNnlRrBrS...|['Francisco Canaro']|
|['2OJEUmIIPsTAUji...|['Radclyffe Hall'...|
|['4AxgXfD7ISvJSTO...|     ['Mistinguett']|
+--------------------+--------------------+
only showing top 10 rows



In [16]:
# Import struct fields that we can use
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

Drop duplicates and incomplete rows

Write DataFrame to RDS

In [17]:
 # Configuration for RDS instance
mode = "append"
jdbc_url="jdbc:postgresql://spotify-db.c3uki6gnymem.us-east-2.rds.amazonaws.com:5432/spotify"
config = {"user":"root", 
          "password": "AWS2021!", 
          "driver":"org.postgresql.Driver"}

In [19]:
tracks_df.write.jdbc(url=jdbc_url, table="tracks_df", mode=mode, properties=config)

Py4JJavaError: ignored